In [61]:
import pandas as pd

crime_raw = pd.read_excel('/Users/souhil/Desktop/Courses/Recherche Empirique/Final/Data/raw/crimes_subject_to_law_raw.xlsx')
pop_df = pd.read_excel(
    '/Users/souhil/Desktop/Courses/Recherche Empirique/Final/Data/raw/population_raw.xlsx', sheet_name='px-x-0102010000_101', header=None, skiprows=2)

# Whole Switzerland

Population dataset

In [62]:
# Clean the population dataset

# Forward-fill missing years
pop_df[0] = pop_df[0].ffill()

# Filter and rename columns
pop_clean = pop_df[pop_df[5].isin(["Suisse", "Étranger"])][[0, 5, 6]] 

pop_clean = pop_clean.rename(columns={0: "year", 5: "group", 6: "population"})

# Map to English categories and clean data types
pop_clean["group"] = pop_clean["group"].map({"Suisse": "Swiss", "Étranger": "Foreigner"})

pop_clean = pop_clean.dropna()
pop_clean["year"] = pop_clean["year"].astype(int)
pop_clean["population"] = pd.to_numeric(pop_clean["population"])

pop_clean.head()

,year,group,population
0,2010,Swiss,6103857.0
1,2010,Foreigner,1837112.0
2,2011,Swiss,6138668.0
3,2011,Foreigner,1896723.0
4,2012,Swiss,6169091.0


Crime subject to law dataset

In [63]:
# Clean crime subject to law dataset

# Extract years
years = crime_raw.iloc[1, 4:].tolist() 
print(years)

# Define group mapping for aggregation
group_mapping = {"suisses": "Swiss", "résidente": "Foreigner", "asile": "Foreigner", "autres": "Foreigner"}

# Process crime data
records = []
for row_idx in range(2, crime_raw.shape[0]):
    label = str(crime_raw.iloc[row_idx, 3]).lower()
    
    # Check if row contains group data
    for french_term in group_mapping:
        if french_term in label:
            group = group_mapping[french_term]
            values = crime_raw.iloc[row_idx, 4:4+len(years)]
            
            # Convert values raising errors to 0 
            counts = pd.to_numeric(values, errors='coerce').fillna(0)
            
            # Add to records
            for year, count in zip(years, counts):
                records.append({"year": int(year), "group": group, "count": count})
            break 


# Create aggregated crime DataFrame
crimes_agg = pd.DataFrame(records)
print(crimes_agg.columns)

crimes_agg = crimes_agg.groupby(['year', 'group'])['count'].sum().reset_index()
crimes_agg.head()

['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
Index(['year', 'group', 'count'], dtype='object')


,year,group,count
0,2009,Foreigner,4654.0
1,2009,Swiss,4238.0
2,2010,Foreigner,4450.0
3,2010,Swiss,4047.0
4,2011,Foreigner,4197.0


In [64]:
# Merge crime subject to law with population data
final_df = pd.merge(
    crimes_agg,
    pop_clean,
    on=['year', 'group'],
    how='left')

# Calculate crime rate
final_df['crime_rate_per_100k'] = (final_df['count'] / final_df['population']) * 100_000
final_df['crime_rate_per_100k'] = final_df['crime_rate_per_100k'].round(2)

# Remove 2024 if population data unavailable
final_df = final_df[final_df['year'] <= 2023]

final_df.head()

,year,group,count,population,crime_rate_per_100k
0,2009,Foreigner,4654.0,NaN,NaN
1,2009,Swiss,4238.0,NaN,NaN
2,2010,Foreigner,4450.0,1837112.0,242.23
3,2010,Swiss,4047.0,6103857.0,66.30
4,2011,Foreigner,4197.0,1896723.0,221.28


In [65]:
# Extract the cleaned relevant crime dataset
final_df.to_excel("/Users/souhil/Desktop/Courses/Recherche Empirique/Final/Data/processed/crimes_subject_to_law_cleaned.xlsx", index=False)

# Cantons datasets

Population by canton dataset

In [66]:
pop_raw = pd.read_excel('/Users/souhil/Desktop/Courses/Recherche Empirique/Final/Data/raw/population_raw_by_canton.xlsx', header=None)


pop_clean = pop_raw.copy()
pop_clean[[0, 1]] = pop_clean[[0, 1]].ffill()
pop_clean.columns = ['year', 'canton', 'group', 'population']

pop_clean = pop_clean[pop_clean['group'].isin(['Suisse', 'Étranger'])]
pop_clean = pop_clean.dropna(subset=['population'])

pop_clean['group'] = pop_clean['group'].map({'Suisse': 'Swiss', 'Étranger': 'Foreigner'})
pop_clean['canton'] = pop_clean['canton'].str.replace("-", "", regex=False).str.strip()
pop_clean['year'] = pop_clean['year'].astype(int)
pop_clean['population'] = pd.to_numeric(pop_clean['population'], errors='coerce')

df_pop = pop_clean[['year', 'canton', 'group', 'population']]

df_pop.head()
#good until here

,year,canton,group,population
3,2010,Zürich,Swiss,1042311
4,2010,Zürich,Foreigner,341629
5,2010,Bern / Berne,Swiss,848054
6,2010,Bern / Berne,Foreigner,138541
7,2010,Luzern,Swiss,314720


Crime subject to the law dataset

In [67]:
# Clean relevant crime data for all cantons

crime_raw = pd.read_excel('/Users/souhil/Desktop/Courses/Recherche Empirique/Final/Data/raw/crimes_subject_to_law_raw_by_canton.xlsx', header=None)

# Extract years from row 2, columns 6+
years = crime_raw.iloc[2, 6:].tolist()

group_map = {
    'suisses': 'Swiss',
    'résidente': 'Foreigner',
    'asile': 'Foreigner',
    'autres': 'Foreigner'
}

records = []

for i in range(3, crime_raw.shape[0], 4):  # every 4 rows = 1 canton
    try:
        canton = crime_raw.iloc[i, 3]
        if pd.isna(canton):
            continue
        for j in range(4):  # Swiss / résidente / asile / autres
            row = crime_raw.iloc[i + j]
            label = str(row[5]).lower()
            for key in group_map:
                if key in label:
                    group = group_map[key]
                    values = row[6:].values
                    for k, year in enumerate(years):
                        try:
                            val = values[k]
                            val = np.nan if str(val).strip().upper() in ["X", "*", ""] else float(val)
                            records.append({
                                "year": int(year),
                                "canton": canton,
                                "group": group,
                                "crime_count": val
                            })
                        except:
                            continue
                    break
    except:
        continue

df_crime_by_cantons = pd.DataFrame(records)
df_clean = (df_crime_by_cantons.groupby(['year', 'canton', 'group'], as_index=False).agg({'crime_count': 'sum'}))
df_clean.head()

,year,canton,group,crime_count
0,2009,Aargau,Foreigner,287.0
1,2009,Aargau,Swiss,219.0
2,2009,Appenzell Ausserrhoden,Foreigner,7.0
3,2009,Appenzell Ausserrhoden,Swiss,5.0
4,2009,Appenzell Innerrhoden,Foreigner,0.0


In [68]:
df_merged = pd.merge(df_clean, df_pop, on=['year', 'canton', 'group'], how='inner')
df_merged['crime_rate_per_100k'] = (df_merged['crime_count'] / df_merged['population']) * 100000
df_final = df_merged[['year', 'canton', 'group', 'crime_count', 'population', 'crime_rate_per_100k']]

df_final.to_excel("/Users/souhil/Desktop/Courses/Recherche Empirique/Final/Data/processed/crimes_subject_to_law_cleaned_by_canton.xlsx", index=False)

In [69]:
print('Done')

Done
